# 521153S Deep Learning Final Project

In [1]:
# import necessary packages
import os
import requests
import zipfile
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.io import read_image
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, ResNet18_Weights, ResNet34_Weights, ResNet50_Weights, ResNet101_Weights, ResNet152_Weights, vgg11, vgg13, vgg16, vgg19, VGG11_Weights, VGG13_Weights, VGG16_Weights, VGG19_Weights
import gdown
import urllib
import os
import random
import tarfile
import time
import shutil
import logging as lg
from tqdm.notebook import tqdm
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

No need to run this if there is nood for debugging.

In [ ]:
# Set the logging level
# Possible Levels: DEBUG, INFO,
lg.basicConfig(level=lg.INFO)

Set the device to use GPU if available.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [4]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

Download the necessary the datasets.

In [5]:
# download the dataset
val_url = 'https://drive.google.com/u/0/uc?id=1hSMUMj5IRpf-nQs1OwgiQLmGZCN0KDWl'
train_url = 'https://drive.google.com/u/0/uc?id=107FTosYIeBn5QbynR46YG91nHcJ70whs'
test_url = 'https://drive.google.com/u/0/uc?id=1yKyKgxcnGMIAnA_6Vr2ilbpHMc9COg-v'
eurosat_url = 'https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1'
cub_url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'

val_file = './data/val.tar'
train_file = './data/train.tar'
test_file = './data/test.tar'
eurosat_file = './data/eurosatrgb.zip'
cub_file = './data/cub_200_2011.tgz'


if not os.path.exists('./data'):
    print('Creating data directory')
    os.mkdir('./data')

if not os.path.exists('./data/val.tar'):
    print('Downloading val.tar')
    gdown.download(val_url, val_file)
    val_tar = tarfile.open(val_file)
    val_tar.extractall('./data/')
    val_tar.close()

if not os.path.exists(train_file):
    print('Downloading train.tar')
    gdown.download(train_url, train_file)
    train_tar = tarfile.open(train_file)
    train_tar.extractall('./data/')
    train_tar.close()

if not os.path.exists(test_file):
    print('Downloading test.tar')
    gdown.download(test_url, test_file)
    test_tar = tarfile.open(test_file)
    test_tar.extractall('./data/')
    test_tar.close()

if not os.path.exists(eurosat_file):
    print('Downloading EuroSAT_RGB.zip')
    response = urllib.request.urlretrieve(eurosat_url, eurosat_file)
    eurosat_zip = zipfile.ZipFile(eurosat_file)
    eurosat_zip.extractall('./data/eurosat')
    eurosat_zip.close()

if not os.path.exists(cub_file):
    print('Downloading cub200_2011.tgz')
    response = urllib.request.urlretrieve(cub_url, cub_file)
    tar = tarfile.open(cub_file, 'r')
    for item in tar:
        tar.extract(item, './data')
    tar.close()


Download the pretrained models.

In [6]:
model_resnet18 = resnet18(weights=ResNet18_Weights.DEFAULT)
model_resnet34 = resnet34(weights=ResNet34_Weights.DEFAULT)
model_resnet50 = resnet50(weights=ResNet50_Weights.DEFAULT)
model_resnet101 = resnet101(weights=ResNet101_Weights.DEFAULT)
model_resnet152 = resnet152(weights=ResNet152_Weights.DEFAULT)

model_vgg11 = vgg11(weights=VGG11_Weights.DEFAULT)
model_vgg13 = vgg13(weights=VGG13_Weights.DEFAULT)
model_vgg16 = vgg16(weights=VGG16_Weights.DEFAULT)
model_vgg19 = vgg19(weights=VGG19_Weights.DEFAULT)

In [7]:
# Stratified split, tries to maintain a proportional distribution of samples for each class in all three sets (train, val and test).

# Val and Test ratio total, Train ratio (1-test_ratio)
test_ratio = 0.3

# Split between Val and Test
test_val_split = 0.5

# Loading the ImageFolder dataset
train_dataset = torchvision.datasets.ImageFolder(root='./data/train')

# Getting the number of classes
num_classes = len(train_dataset.classes)

# Initializing lists to store indices for train, validation, and test sets
train_indices, val_indices, test_indices = [], [], []

# Splitting each class separately
for class_index in range(num_classes):
    # Getting indices for samples belonging to the current class
    class_indices = np.where(np.array(train_dataset.targets) == class_index)[0]
    
    # Splitting indices for each class separately into train, validation, and test sets
    train_idx, temp_idx = train_test_split(class_indices, test_size=test_ratio, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=test_val_split, random_state=42)
    
    # Extending the lists to store indices for each set
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)
    test_indices.extend(test_idx)

# Creating Subset objects using the selected indices for each set
train_set = Subset(train_dataset, train_indices)
val_set = Subset(train_dataset, val_indices)
test_set = Subset(train_dataset, test_indices)

# Transform functions
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),      
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

train_set.dataset.transform = transform_train
val_set.dataset.transform = transform_eval
test_set.dataset.transform = transform_eval

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Printing the sizes of the resulting sets
print(f"Number of samples in the training set: {len(train_set)}")
print(f"Number of samples in the validation set: {len(val_set)}")
print(f"Number of samples in the test set: {len(test_set)}")

Number of samples in the training set: 26880
Number of samples in the validation set: 5760
Number of samples in the test set: 5760


In [7]:
# Define the used model
model = model_resnet18
model.name = "resnet18"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [9]:
def evaluate(model, data_loader, device, criterion):

    model.eval() 
    valid_running_correct = 0
    counter = 0
    num_images = 0.0 # used to accumulate number of images
    
    for i, data in enumerate(data_loader, 0):
        counter += 1
        
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = criterion(outputs, labels)
        # Calculate the accuracy.
        preds = torch.max(outputs, 1).indices
        valid_running_correct += torch.sum(preds == labels.data)
        num_images += len(labels)
        
    # Loss and accuracy for the complete epoch.
    lg.debug(f"EVAL:   loss.item(): {loss.item()} valid_running_correct: {valid_running_correct.item()}")

    epoch_acc = valid_running_correct / num_images
    return epoch_acc

In [11]:
def train(model, trainLoader, valLoader, criterion, optimizer, num_epochs, device):
    model.to(device)

    # Start the training.
    if (torch.cuda.is_available() & device.type == 'cuda'):
        print("Using CUDA")

    print('Training')
    
    lg.debug(f" len(trainLoader.dataset): {len(trainLoader.dataset)}")
    lg.debug(f" len(valLoader.dataset): {len(valLoader.dataset)}")

    
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        num_images = 0.0 # used to accumulate number of images
        for i, data in enumerate(trainLoader,0):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += torch.sum(preds == labels.data)
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()    # Output training 
            num_images += len(image)
            if counter % 100 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Step: {counter}/{len(trainLoader)}, Loss: {loss.item():.4f}')
        
        # # reduce learning rate every 10 epochs by factor of 10
        # if (epoch+1) % 10 == 0:
        #     for param_group in optimizer.param_groups:
        #         param_group['lr'] /= 10.0


        # Validation
        valid_epoch_acc = evaluate(model, valLoader, device, criterion)
        lg.debug(f"TRAIN: loss.item(): {loss.item()}, train_running_correct: {train_running_correct}")
        
        train_epoch_acc = train_running_correct  / num_images
        print('Epoch: %d/%d, Training accuracy: %f, Loss: %f, Validation accuracy: %f' % (epoch+1, num_epochs, train_epoch_acc, loss.item(), valid_epoch_acc))
        print('-'*50)

    return model

In [11]:
pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Using CUDA
Training
Epoch: 1/10, Step: 100/210, Loss: 2.8664
Epoch: 1/10, Step: 200/210, Loss: 1.6076
Epoch: 1/10, Training accuracy: 0.417820, Loss: 1.677926, Validation accuracy: 0.699479
--------------------------------------------------
Epoch: 2/10, Step: 100/210, Loss: 1.2053
Epoch: 2/10, Step: 200/210, Loss: 0.9176
Epoch: 2/10, Training accuracy: 0.759747, Loss: 1.067810, Validation accuracy: 0.783681
--------------------------------------------------
Epoch: 3/10, Step: 100/210, Loss: 0.8008
Epoch: 3/10, Step: 200/210, Loss: 0.7590
Epoch: 3/10, Training accuracy: 0.818638, Loss: 0.829025, Validation accuracy: 0.813194
--------------------------------------------------
Epoch: 4/10, Step: 100/210, Loss: 0.5513
Epoch: 4/10, Step: 200/210, Loss: 0.6269
Epoch: 4/10, Training accuracy: 0.853981, Loss: 0.520096, Validation accuracy: 0.831771
--------------------------------------------------
Epoch: 5/10, Step: 100/210, Loss: 0.4291
Epoch: 5/10, Step: 200/210, Loss: 0.6209
Epoch: 5/10, T

In [12]:
def save_model(trained_model):
    # Create Models Folder
    if not os.path.exists('./models'):
        print('Creating models directory')
        os.mkdir('./models')

    # Save the model
    if not os.path.exists(f'./models/{trained_model.name}.pth'):
        print(f'Saving Model {trained_model.name}')
        torch.save(trained_model, f'./models/{trained_model.name}.pth')

def load_model(model_name):
    if not os.path.exists(f'./models/{model_name}.pth'):
        print(f'Cannot Load {model_name}')
        return None
    print(f'Loading Model {model_name}')
    model = torch.load(f'./models/{model_name}.pth')
    model.eval()
    return model

Save the pre-trained model after training

In [16]:
save_model(pretrained_model)

Saving Model resnet18


Evalue the model on test set

In [14]:
test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

test acc: 0.846


# EuroSAT dataset

Choose the 100 images from EuroSAT dataset for the training and validation.

In [13]:
def choose_100_eurosat():
    '''
    The `choose_100_eurosat()` function selects 100 images from the EuroSAT dataset and copies them to the `data/eurosat_validation` folder. It also creates a `data/eurosat_training` folder if it doesn't already exist.
    '''
    if not os.path.exists('./data/eurosat_validation'):
        print('Creating data directory')
        os.mkdir('./data/eurosat_validation')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_validation'))
        os.mkdir('./data/eurosat_validation')

    if not os.path.exists('./data/eurosat_training'):
        print('Creating training directory')
        os.mkdir('./data/eurosat_training')
    else:
        # Folder full of images -> delete all files in the directory
        shutil.rmtree(os.path.join('./data/eurosat_training'))
        os.mkdir('./data/eurosat_training')

    # Load the EuroSAT Categories
    eurosat_categories = [name for name in os.listdir('./data/eurosat/EuroSAT_RGB/') if os.path.isdir(os.path.join('./data/eurosat/EuroSAT_RGB/', name))]
    # Randomly select 5 categories
    lg.debug(f"Selecting 5 categories from {eurosat_categories} categories")
    selected_categories = np.random.choice(eurosat_categories, 5, replace=False)
    lg.debug(f"Selected categories: {selected_categories}")

    selected_images = []
    training_images = []

    # From each directory, randomly select 20 images
    for category in selected_categories:
        images = os.listdir(os.path.join('./data/eurosat/EuroSAT_RGB/', category))
        selected = random.sample(images, 20)
        selected_images.extend([(category, image) for image in selected])
        lg.debug(f"Selected {selected} from {category}")
        # Copy selected images to the selected directory
        for image in selected:
            if not os.path.exists(f"./data/eurosat_validation/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_validation/{category}")
            shutil.copyfile(os.path.join('./data/eurosat/EuroSAT_RGB', category, image), os.path.join(f"./data/eurosat_validation/{category}", image))

    # From these 100 images, randomly select 5 images from each category for the training set
    for category in selected_categories:
        category_images = [image for (cat, image) in selected_images if cat == category]
        training = random.sample(category_images, 5)
        training_images.extend(training)
        lg.debug(f"Selected {training} for training")

        # Copy training images to the training directory
        for image in training:
            if not os.path.exists(f"./data/eurosat_training/{category}"):
                lg.debug(f"Creating {category} directory")
                os.mkdir(f"./data/eurosat_training/{category}")
            shutil.move(os.path.join(f"./data/eurosat_validation/{category}", image), os.path.join(f"./data/eurosat_training/{category}", image))

In [14]:
batch_size = 12

In [15]:
def load_euro_datasets():

    # Choose 100 images from EuroSAT dataset
    choose_100_eurosat()

    eurosat_train_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_training',
        transform=transforms.Compose([
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
        ])
    )

    eurosat_validation_dataset = torchvision.datasets.ImageFolder(
        root='./data/eurosat_validation',
        transform=transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    )

    eurosat_dataloader_train = DataLoader(eurosat_train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    eurosat_dataloader_validation = DataLoader(eurosat_validation_dataset, batch_size=batch_size, shuffle = False, num_workers=num_workers)

    return eurosat_dataloader_train, eurosat_dataloader_validation

Train the model on EuroSAT training dataset and validate on the EuroSAT validation dataset.

In [20]:
# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False

model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet18_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Load the EuroSAT dataset
eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()

if os.path.exists("results.txt"):
    # If the file exists, clear its contents
    with open("results.txt", "w") as f:
        f.truncate(0)  # Clear the file content
        print("File 'results.txt' found and cleared.")

# Train and Evaluate the model on the EuroSAT dataset
i = 5
average_results = 0.0

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    model_resnet18_fine = train(model_resnet18_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(model_resnet18_fine, eurosat_dataloader_validation, device, criterion)
    average_results += results
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{model_resnet18_fine.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")

average_results /= i
print(f"Average results: {average_results:.3f}")
with open("results.txt", "a") as f:
        f.write(f"{model_resnet18_fine.name} Average results: {average_results:.6f}\n")

Loading Model resnet18
File 'results.txt' found and cleared.
Training EuroSAT model 1/5
Using CUDA
Training
Epoch: 1/30, Training accuracy: 0.360000, Loss: 1.455138, Validation accuracy: 0.253333
--------------------------------------------------
Epoch: 2/30, Training accuracy: 0.320000, Loss: 0.856582, Validation accuracy: 0.213333
--------------------------------------------------
Epoch: 3/30, Training accuracy: 0.200000, Loss: 2.561901, Validation accuracy: 0.226667
--------------------------------------------------
Epoch: 4/30, Training accuracy: 0.240000, Loss: 1.166750, Validation accuracy: 0.213333
--------------------------------------------------
Epoch: 5/30, Training accuracy: 0.240000, Loss: 0.437903, Validation accuracy: 0.200000
--------------------------------------------------
Epoch: 6/30, Training accuracy: 0.200000, Loss: 2.344654, Validation accuracy: 0.360000
--------------------------------------------------
Epoch: 7/30, Training accuracy: 0.440000, Loss: 1.855628, 

## RESNET 34

In [16]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

# Define the used model
model = model_resnet34
model.name = "resnet34"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

save_model(pretrained_model)

test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

batch_size = 12

# Load the pretrained model
model_resnet34_fine = load_model("resnet34")

for param in model_resnet34_fine.parameters():
    param.requires_grad = False

model_resnet34_fine.fc = nn.Linear(model_resnet34_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet34_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Train and Evaluate the model on the EuroSAT dataset
i = 5
average_results = 0.0

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    model_resnet34_fine = train(model_resnet34_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(model_resnet34_fine, eurosat_dataloader_validation, device, criterion)
    average_results += results
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{model_resnet34_fine.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")
        # f.close()

average_results /= i
print(f"Average results: {average_results:.3f}")
with open("results.txt", "a") as f:
        f.write(f"{model_resnet34_fine.name} Average results: {average_results:.6f}\n")
        # f.close()


Using CUDA
Training
Epoch: 1/10, Step: 100/210, Loss: 2.5538
Epoch: 1/10, Step: 200/210, Loss: 1.3538
Epoch: 1/10, Training accuracy: 0.478348, Loss: 1.575795, Validation accuracy: 0.769097
--------------------------------------------------


## RESNET 50

In [ ]:
# Image Size
image_size = 224

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers (id not good, decrease learning rate (try 0.001, 0.0001, 0.00001))
lr = 0.001

# Momentum
momentum = 0.9

# Number of training epochs
num_epochs = 10

# Weight decay (if not good, increase weight_decay (try 1e-4, 1e-3, 1e-2))
weight_decay=1e-2

# Define the used model
model = model_resnet50
model.name = "resnet50"     # Set name for saving

model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

pretrained_model = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

save_model(pretrained_model)

test_acc = evaluate(pretrained_model, test_loader, device, criterion)
print(f"test acc: {test_acc:.3f}")

batch_size = 12

# Load the pretrained model
model_resnet50_fine = load_model("resnet50")

for param in model_resnet50_fine.parameters():
    param.requires_grad = False

model_resnet50_fine.fc = nn.Linear(model_resnet50_fine.fc.in_features, 5)

# Define loss function and optimizer for fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet50_fine.parameters(), lr=0.001, momentum=0.9)

num_epochs = 30

# Train and Evaluate the model on the EuroSAT dataset
i = 5
average_results = 0.0

for j in range(i):
    print(f"Training EuroSAT model {j+1}/{i}")
    eurosat_dataloader_train, eurosat_dataloader_validation = load_euro_datasets()
    model_resnet50_fine = train(model_resnet50_fine, eurosat_dataloader_train, eurosat_dataloader_validation, criterion, optimizer, num_epochs, device)
    results = evaluate(model_resnet50_fine, eurosat_dataloader_validation, device, criterion)
    average_results += results
    print(f"EuroSAT model {j+1}/{i} results: {results:.3f}")
    # Save the results to results.txt
    with open("results.txt", "a") as f:
        f.write(f"{model_resnet50_fine.name} {j} Time: {time.strftime('%d/%m/%Y, %H:%M:%S')} Epochs: {num_epochs} | Results: {results:.6f}\n")

average_results /= i
print(f"Average results: {average_results:.3f}")
with open("results.txt", "a") as f:
        f.write(f"{model_resnet50_fine.name} Average results: {average_results:.6f}\n")


## DATASET: CUB

In [15]:
batch_size = 16

cub_dataset = torchvision.datasets.ImageFolder(root='./data/CUB_200_2011/images')

# Transform functions
transform_eval = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), # mean and std for ImageNet dataset
])

cub_dataset.transform = transform_eval

# Getting the number of classes
cub_num_classes = len(cub_dataset.classes)

cub_val_loader = DataLoader(cub_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Printing the sizes of the resulting sets
print(f"Number of samples in the validation set: {len(cub_dataset)}")
print(f"Number of classes in CUB-200: {cub_num_classes}")


Number of samples in the validation set: 11788
Num classes: 200


In [17]:
# Load the pretrained model
model_resnet18_fine = load_model("resnet18")

for param in model_resnet18_fine.parameters():
    param.requires_grad = False

model_resnet18_fine.fc = nn.Linear(model_resnet18_fine.fc.in_features, cub_num_classes)     # Change the fc layer to reflect the number of classes in CUB dataset
criterion = nn.CrossEntropyLoss()       # Define loss function and optimizer for fine-tuning
device = torch.device('cpu')
model_resnet18_fine.to(device)

if (torch.cuda.is_available() & device.type == 'cuda'):
    print("Using CUDA")

cub_model_acc = evaluate(model_resnet18_fine, cub_val_loader, device, criterion)
print(f"CUB model results: {cub_model_acc:.3f}")

Loading Model resnet18
CUB model results: 0.008
